In [8]:
import psycopg2
import os
from dotenv import load_dotenv
import uuid
import pandas as pd

# Load environment variables
load_dotenv()

# Database connection parameters
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")  # Ensure you have DB_PORT in your .env file

csv_file_path = '../data/iucn_categories.csv'

# Read CSV file into DataFrame
df_categories = pd.read_csv(csv_file_path)

print(df_categories)

# Establishing Database Connection
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cursor = conn.cursor()

    # Iterate over DataFrame rows
    for index, row in df_categories.iterrows():

      # Ensure all values are strings and escape single quotes
      name = str(row['name']).replace("'", "''") if pd.notna(row['name']) else ''
      abbreviation = str(row['abbreviation']).replace("'", "''") if pd.notna(row['abbreviation']) else ''
      description = str(row['description']).replace("'", "''") if pd.notna(row['description']) else ''

      # Generate UUID for each category
      iucn_category_id = uuid.uuid4()

      # Prepare the SQL INSERT statement
      sql_statement = f"""
      INSERT INTO iucn_categories (iucn_category_id, name, abbreviation, description, created, updated)
      VALUES ('{iucn_category_id}', '{name}', '{abbreviation}', '{description}', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
      """

      # Execute the SQL statement
      cursor.execute(sql_statement)

    # Commit the transaction
    conn.commit()
    print("All records inserted successfully")

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    if conn:
        cursor.close()
        conn.close()

                    name abbreviation   
0  Critically Endangered           CR  \
1         Data Deficient           DD   
2             Endangered           EN   
3    Extinct In The Wild           EW   
4                Extinct           EX   
5          Least Concern           LC   
6        Near Threatened           NT   
7          Not Evaluated           NE   
8             Vulnerable           VU   

                                         description  
0  A taxon is Critically Endangered when the best...  
1  A taxon is Data Deficient when there is inadeq...  
2  A taxon is Endangered when the best available ...  
3  A taxon is Extinct in the Wild when it is know...  
4  A taxon is Extinct when there is no reasonable...  
5  A taxon is Least Concern when it has been eval...  
6  A taxon is Near Threatened when it has been ev...  
7  A taxon is Not Evaluated when it is has not ye...  
8  A taxon is Vulnerable when the best available ...  
All records inserted successfully
